In [ ]:
import os, time, shutil, sys
from pathlib import Path
from pathlib import Path
from glob import glob
import h5py
import numpy as np
import spikeinterface.full as si
sys.path.append("/home/phornauer/Git/axon_tracking/")
from axon_tracking import spike_sorting as ss

In [ ]:
sorter = 'kilosort2_5'
si.Kilosort2_5Sorter.set_kilosort2_5_path('/home/phornauer/Git/Kilosort_2020b')
sorter_params = si.get_default_sorter_params(si.Kilosort2_5Sorter)
sorter_params['n_jobs'] = -1
sorter_params['detect_threshold'] = 6
sorter_params['minFR'] = 0.01
sorter_params['minfr_goodchannels'] = 0.01
sorter_params['keep_good_only'] = False
sorter_params['do_correction'] = False

In [ ]:
N_recordings = 3 # Number of recordings per chip/well that need to exist
parent_path = '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/tagamma/'
recording_date = '231019'
project_name = 'DCZ_*2'
chip_id = 'T002443'

assay_name = 'Network'
assay_id = '0*'
file_name = 'data.raw.h5'

path_parts = [parent_path, recording_date, project_name, chip_id, assay_name, assay_id, file_name]
path_pattern = os.path.join(*path_parts)
path_list  = glob(path_pattern)

Optional: Manipulate the recording list if needed

In [ ]:
path_list.sort()
recording_path_list = path_list
#recording_path_list.append('/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/nobenz/Chemo_hippo3/230919/M05569/Network/000053/data.raw.h5')
recording_path_list

In [ ]:
recording_path_list = [recording_path_list[x] for x in [0,1,2,3,5]]

Or specify the recording file list yourself entirely

In [ ]:
recording_path_list = ['/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/Practicals/Group5/0000/Network/000111/data.raw.h5',
                      '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/Practicals/Group5/0000/Network/000114/data.raw.h5']

save_root = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/Practicals/Group5/0000/concatenated/'

Optional: Check if the stream ids are correct

In [ ]:
h5 = h5py.File(recording_path_list[-1])
stream_ids = list(h5['wells'].keys())
stream_ids

In [ ]:
for stream_id in stream_ids:
    
    well_recording_list = []
    if not os.path.exists(os.path.join(save_root, stream_id,'sorter_output','amplitudes.npy')):
        for rec_path in recording_path_list: #Iterate over recordings to be concatenated 
            
            try: # If not all wells were recorded, should be the only cause for an error
                rec = si.MaxwellRecordingExtractor(rec_path, stream_id=stream_id)
                well_recording_list.append(rec)
            except Exception:
                continue
        
        if len(well_recording_list) >= len(recording_path_list):
            multirecording = si.concatenate_recordings(well_recording_list) #[:N_recordings]
            ss.clean_sorting(rec=multirecording, save_root=save_root, stream_id=stream_id, sorter=sorter, sorter_params=sorter_params)